In [1]:
import datetime
import time
import os

import numpy as np

import keras
import tensorflow as tf
from keras.layers import BatchNormalization, Conv2D, Dense, Dropout, GlobalAveragePooling2D, Input, MaxPooling2D, Rescaling
from keras.models import Model, Sequential

from tensorboard.plugins.hparams import api as hp

In [2]:
def set_seed(s):
    # https://www.tensorflow.org/api_docs/python/tf/config/experimental/enable_op_determinism
    tf.config.experimental.enable_op_determinism()

    # https://www.tensorflow.org/api_docs/python/tf/keras/utils/set_random_seed
    tf.keras.utils.set_random_seed(s)

    os.environ["PYTHONHASHSEED"] = str(s)
    NP_RANDOM = np.random.default_rng(s)

In [3]:
SEED_1 = 424242
SEED_2 = 314159
SEED_3 = 168740
SEED_4 = 856263
SEED_5 = 241223
set_seed(SEED_1)

In [4]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [5]:
!nvidia-smi

Sat Jan 27 18:42:08 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
DATA_DIR = "/content/drive/MyDrive/CAS_Studienarbeit_Data/Wildfire/train"
VAL_DIR = "/content/drive/MyDrive/CAS_Studienarbeit_Data/Wildfire/val"
TEST_DIR = "/content/drive/MyDrive/CAS_Studienarbeit_Data/Wildfire/test"

In [8]:
EPOCHS = 50
BATCH_SIZE = 32
IMG_SIZE = (150, 150)

In [9]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    label_mode='binary',
    seed=SEED_1,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE)

val_ds = tf.keras.utils.image_dataset_from_directory(
    VAL_DIR,
    label_mode='binary',
    seed=SEED_1,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE)

test_ds = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    label_mode='binary',
    seed=SEED_1,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE)

Found 1887 files belonging to 2 classes.
Found 401 files belonging to 2 classes.
Found 410 files belonging to 2 classes.


In [10]:
print(train_ds.class_names)
print(val_ds.class_names)
print(test_ds.class_names)

['fire', 'nofire']
['fire', 'nofire']
['fire', 'nofire']


In [11]:
train_ds = train_ds.cache().prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.cache().prefetch(tf.data.AUTOTUNE)

In [12]:
HP_CONV_LAYERS = hp.HParam("conv_layers", hp.IntInterval(1, 5))
HP_CONV_FILTERS = hp.HParam("conv_filters", hp.Discrete([16, 32, 64, 128]))
HP_DROPOUT_TYPE = hp.HParam(
    "dropout", hp.Discrete(["none", "single", "multiple"]))
HP_BATCH_NORM = hp.HParam("bn", hp.Discrete([True, False]))

HPARAMS = [
    HP_CONV_LAYERS,
    HP_CONV_FILTERS,
    HP_DROPOUT_TYPE,
    HP_BATCH_NORM
]

METRICS = [
    hp.Metric(
        "train/loss",
        group="custom-metrics",
        display_name="loss (train)",
    ),
    hp.Metric(
        "train/accuracy",
        group="custom-metrics",
        display_name="accuracy (train)",
    ),
    hp.Metric(
        "train/f1",
        group="custom-metrics",
        display_name="f1 (train)",
    ),
    hp.Metric(
        "validation/loss",
        group="custom-metrics",
        display_name="loss (validation)",
    ),
    hp.Metric(
        "validation/accuracy",
        group="custom-metrics",
        display_name="accuracy (validation)",
    ),
    hp.Metric(
        "validation/f1",
        group="custom-metrics",
        display_name="f1 (validation)",
    ),
    hp.Metric(
        "test/loss",
        group="custom-metrics",
        display_name="loss (test)",
    ),
    hp.Metric(
        "test/accuracy",
        group="custom-metrics",
        display_name="accuracy (test)",
    ),
    hp.Metric(
        "test/f1",
        group="custom-metrics",
        display_name="f1 (test)",
    ),
]

In [13]:
def model_fn(hparams, seed):
    model = tf.keras.models.Sequential()
    model.add(keras.Input(shape=(150, 150, 3), name="image_input"))
    model.add(Rescaling(scale=1./127.5, offset=-1, name="input_normalization"))

    for idx, _ in enumerate(range(hparams[HP_CONV_LAYERS])):
        model.add(
            Conv2D(
                filters=hparams[HP_CONV_FILTERS],
                kernel_size=3,
                padding="same",
                activation="relu",
                name=f"conv_{idx}"
            )
        )
        if hparams[HP_BATCH_NORM]:
            model.add(BatchNormalization(name=f"conv_{idx}_bn"))

        model.add(MaxPooling2D(pool_size=(2, 2),
                  strides=2, name=f"max_pooling_{idx}"))
        if hparams[HP_DROPOUT_TYPE] == "multiple":
            model.add(Dropout(0.5, name=f"dropout_{idx}", seed=seed))

    model.add(GlobalAveragePooling2D(name="avg_pooling"))

    model.add(Dense(16, activation="relu", name=f"dense_final"))

    if hparams[HP_DROPOUT_TYPE] != "none":
        model.add(Dropout(0.5, name=f"dropout_final", seed=seed))

    model.add(Dense(1, activation="sigmoid", name="output"))

    model.compile(
        loss="binary_crossentropy",
        optimizer="adam",
        metrics=["accuracy", tf.keras.metrics.F1Score(
            threshold=0.5, name='f1_score')],
    )
    return model

In [14]:
def run(base_dir, seed, hparams, name):
    """
    Train a CNN with a given parameter config
    """
    model = model_fn(hparams=hparams, seed=seed)
    logdir = os.path.join(base_dir, name)

    tb_callback = tf.keras.callbacks.TensorBoard(
        logdir,
        write_graph=False,
        profile_batch=0,
        update_freq=20
    )
    hparams_callback = hp.KerasCallback(logdir, hparams, trial_id=name)

    results = model.fit(
        train_ds,
        epochs=EPOCHS,
        shuffle=False,
        validation_data=val_ds,
        callbacks=[hparams_callback, tb_callback],
        verbose=1
    )

    return model, results

In [15]:
def run_all(base_dir, seed, verbose=True):
    """
    Perform all runs for a single seed
    """
    logdir = os.path.join(base_dir, str(seed))
    with tf.summary.create_file_writer(logdir).as_default():
        hp.hparams_config(hparams=HPARAMS, metrics=METRICS)

    run_index = 0
    for conv_layers in range(HP_CONV_LAYERS.domain.min_value, HP_CONV_LAYERS.domain.max_value + 1):
        for conv_filters in HP_CONV_FILTERS.domain.values:
            for dropout_type in HP_DROPOUT_TYPE.domain.values:
                for bn in HP_BATCH_NORM.domain.values:
                    hparams = {
                        HP_CONV_LAYERS: conv_layers,
                        HP_CONV_FILTERS: conv_filters,
                        HP_DROPOUT_TYPE: dropout_type,
                        HP_BATCH_NORM: bn
                    }
                    name = f"s{
                        seed}-cl{conv_layers}-cf{conv_filters}-d{dropout_type[0]}-bn{int(bn)}"
                    run_index += 1

                    if os.path.exists(os.path.join(base_dir, 'models', f'{name}@{EPOCHS}.keras')):
                        print(
                            f'{name}@{EPOCHS}.keras exists already and will be skipped')
                        continue

                    if verbose:
                        hparams_string = str(
                            [f"{k}: {v}" for k, v in hparams.items()])
                        print(
                            f"--- Starting training session {run_index} ({run_index/120:.2f}): {name}")
                        print(hparams_string)
                    model, results = run(logdir, seed, hparams, name)

                    test_results = model.evaluate(test_ds)

                    # Save trained model for this run
                    if not os.path.exists(os.path.join(base_dir, 'models')):
                        os.makedirs(os.path.join(base_dir, 'models'))
                    model.save(os.path.join(
                        base_dir, 'models', f'{name}@{EPOCHS}.keras'))

                    # Log training/validation/test metrics to tensorboard
                    writer = tf.summary.create_file_writer(
                        os.path.join(logdir, name, 'custom-metrics'))
                    with writer.as_default():
                        for epoch in range(len(results.history['f1_score'])):
                            tf.summary.scalar(
                                'train/loss', results.history['loss'][epoch], step=epoch)
                            tf.summary.scalar(
                                'train/accuracy', results.history['accuracy'][epoch], step=epoch)
                            tf.summary.scalar(
                                'train/f1', results.history['f1_score'][epoch][0], step=epoch)
                            tf.summary.scalar(
                                'validation/loss', results.history['val_loss'][epoch], step=epoch)
                            tf.summary.scalar(
                                'validation/accuracy', results.history['val_accuracy'][epoch], step=epoch)
                            tf.summary.scalar(
                                'validation/f1', results.history['val_f1_score'][epoch][0], step=epoch)
                        tf.summary.scalar(
                            'test/loss', test_results[0], step=EPOCHS)
                        tf.summary.scalar(
                            'test/accuracy', test_results[1], step=EPOCHS)
                        tf.summary.scalar(
                            'test/f1', test_results[2][0], step=EPOCHS)
                        writer.flush()

In [16]:
# %load_ext tensorboard

In [17]:
# %tensorboard --logdir /content/drive/MyDrive/CAS_Studienarbeit_Data/runs

In [18]:
BASE_DIR = '/content/drive/MyDrive/CAS_Studienarbeit_Data/runs'

In [19]:
run_all(BASE_DIR, SEED_1)

s424242-cl1-cf16-dm-bn0@50.keras exists already and will be skipped
s424242-cl1-cf16-dm-bn1@50.keras exists already and will be skipped
s424242-cl1-cf16-dn-bn0@50.keras exists already and will be skipped
s424242-cl1-cf16-dn-bn1@50.keras exists already and will be skipped
s424242-cl1-cf16-ds-bn0@50.keras exists already and will be skipped
s424242-cl1-cf16-ds-bn1@50.keras exists already and will be skipped
s424242-cl1-cf32-dm-bn0@50.keras exists already and will be skipped
s424242-cl1-cf32-dm-bn1@50.keras exists already and will be skipped
s424242-cl1-cf32-dn-bn0@50.keras exists already and will be skipped
s424242-cl1-cf32-dn-bn1@50.keras exists already and will be skipped
s424242-cl1-cf32-ds-bn0@50.keras exists already and will be skipped
s424242-cl1-cf32-ds-bn1@50.keras exists already and will be skipped
s424242-cl1-cf64-dm-bn0@50.keras exists already and will be skipped
s424242-cl1-cf64-dm-bn1@50.keras exists already and will be skipped
s424242-cl1-cf64-dn-bn0@50.keras exists already 

In [20]:
set_seed(SEED_2)
run_all(BASE_DIR, SEED_2)

s314159-cl1-cf16-dm-bn0@50.keras exists already and will be skipped
s314159-cl1-cf16-dm-bn1@50.keras exists already and will be skipped
s314159-cl1-cf16-dn-bn0@50.keras exists already and will be skipped
s314159-cl1-cf16-dn-bn1@50.keras exists already and will be skipped
s314159-cl1-cf16-ds-bn0@50.keras exists already and will be skipped
s314159-cl1-cf16-ds-bn1@50.keras exists already and will be skipped
s314159-cl1-cf32-dm-bn0@50.keras exists already and will be skipped
s314159-cl1-cf32-dm-bn1@50.keras exists already and will be skipped
s314159-cl1-cf32-dn-bn0@50.keras exists already and will be skipped
s314159-cl1-cf32-dn-bn1@50.keras exists already and will be skipped
s314159-cl1-cf32-ds-bn0@50.keras exists already and will be skipped
s314159-cl1-cf32-ds-bn1@50.keras exists already and will be skipped
s314159-cl1-cf64-dm-bn0@50.keras exists already and will be skipped
s314159-cl1-cf64-dm-bn1@50.keras exists already and will be skipped
s314159-cl1-cf64-dn-bn0@50.keras exists already 

In [21]:
set_seed(SEED_3)
run_all(BASE_DIR, SEED_3)

s168740-cl1-cf16-dm-bn0@50.keras exists already and will be skipped
s168740-cl1-cf16-dm-bn1@50.keras exists already and will be skipped
s168740-cl1-cf16-dn-bn0@50.keras exists already and will be skipped
s168740-cl1-cf16-dn-bn1@50.keras exists already and will be skipped
s168740-cl1-cf16-ds-bn0@50.keras exists already and will be skipped
s168740-cl1-cf16-ds-bn1@50.keras exists already and will be skipped
s168740-cl1-cf32-dm-bn0@50.keras exists already and will be skipped
s168740-cl1-cf32-dm-bn1@50.keras exists already and will be skipped
s168740-cl1-cf32-dn-bn0@50.keras exists already and will be skipped
s168740-cl1-cf32-dn-bn1@50.keras exists already and will be skipped
s168740-cl1-cf32-ds-bn0@50.keras exists already and will be skipped
s168740-cl1-cf32-ds-bn1@50.keras exists already and will be skipped
s168740-cl1-cf64-dm-bn0@50.keras exists already and will be skipped
s168740-cl1-cf64-dm-bn1@50.keras exists already and will be skipped
s168740-cl1-cf64-dn-bn0@50.keras exists already 

In [22]:
set_seed(SEED_4)
run_all(BASE_DIR, SEED_4)

s856263-cl1-cf16-dm-bn0@50.keras exists already and will be skipped
s856263-cl1-cf16-dm-bn1@50.keras exists already and will be skipped
s856263-cl1-cf16-dn-bn0@50.keras exists already and will be skipped
s856263-cl1-cf16-dn-bn1@50.keras exists already and will be skipped
s856263-cl1-cf16-ds-bn0@50.keras exists already and will be skipped
s856263-cl1-cf16-ds-bn1@50.keras exists already and will be skipped
s856263-cl1-cf32-dm-bn0@50.keras exists already and will be skipped
s856263-cl1-cf32-dm-bn1@50.keras exists already and will be skipped
s856263-cl1-cf32-dn-bn0@50.keras exists already and will be skipped
s856263-cl1-cf32-dn-bn1@50.keras exists already and will be skipped
s856263-cl1-cf32-ds-bn0@50.keras exists already and will be skipped
s856263-cl1-cf32-ds-bn1@50.keras exists already and will be skipped
s856263-cl1-cf64-dm-bn0@50.keras exists already and will be skipped
s856263-cl1-cf64-dm-bn1@50.keras exists already and will be skipped
s856263-cl1-cf64-dn-bn0@50.keras exists already 

In [23]:
set_seed(SEED_5)
run_all(BASE_DIR, SEED_5)

s241223-cl1-cf16-dm-bn0@50.keras exists already and will be skipped
s241223-cl1-cf16-dm-bn1@50.keras exists already and will be skipped
s241223-cl1-cf16-dn-bn0@50.keras exists already and will be skipped
s241223-cl1-cf16-dn-bn1@50.keras exists already and will be skipped
s241223-cl1-cf16-ds-bn0@50.keras exists already and will be skipped
s241223-cl1-cf16-ds-bn1@50.keras exists already and will be skipped
s241223-cl1-cf32-dm-bn0@50.keras exists already and will be skipped
s241223-cl1-cf32-dm-bn1@50.keras exists already and will be skipped
s241223-cl1-cf32-dn-bn0@50.keras exists already and will be skipped
s241223-cl1-cf32-dn-bn1@50.keras exists already and will be skipped
s241223-cl1-cf32-ds-bn0@50.keras exists already and will be skipped
s241223-cl1-cf32-ds-bn1@50.keras exists already and will be skipped
s241223-cl1-cf64-dm-bn0@50.keras exists already and will be skipped
s241223-cl1-cf64-dm-bn1@50.keras exists already and will be skipped
s241223-cl1-cf64-dn-bn0@50.keras exists already 